# Initialisation and Dependencies

In [1]:
import sys
lib_path = '/home/jovyan/libs'
sys.path.insert(0, lib_path)

In [26]:
%reload_ext autoreload
%autoreload 2

import gc, math, traceback, datetime

import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from datasets import load_from_disk

import whisper
from whisper.tokenizer import get_tokenizer

from utils import audio, gradient, gpu
from utils.attacks import PrepareFront, PrepareAtPosition

# GPU RAM Tracking

In [8]:
device = gpu.get_device()

Device: cuda


# Load Model

In [9]:
try:
    del model
    print("Model deleted!")
except:
    pass

In [10]:
MODEL_NAME = "tiny.en"

model = whisper.load_model(MODEL_NAME).to(device)
model.eval();

/home/jovyan/libs/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


# Load Data

In [11]:
tedlium_path = "../tedlium"
train_path, validation_path, test_path = f"{tedlium_path}/train_idx.hf", f"{tedlium_path}/validation_idx.hf", f"{tedlium_path}/test.hf"

In [12]:
TRAIN_SELECT = 500
VALID_SELECT = 150
TEST_SELECT = 250

SEED = 1

tedlium_train = load_from_disk(train_path).with_format("torch").shuffle(seed=SEED).select(range(TRAIN_SELECT))
tedlium_validation = load_from_disk(validation_path).with_format("torch").shuffle(seed=SEED).select(range(VALID_SELECT))
tedlium_test = load_from_disk(test_path).with_format("torch").shuffle(seed=SEED).select(range(TEST_SELECT))

Loading dataset from disk:   0%|          | 0/109 [00:00<?, ?it/s]

In [13]:
# def collate(ls):
#     pad_to = max(list(map(lambda x: x["audio"].shape[0], ls)))
#     return torch.cat(list(map(lambda x: F.pad(x["audio"], (0, pad_to - x["audio"].shape[0])).unsqueeze(0).to(torch.bfloat16), ls)), dim=0)

def collate_idx(ls):
    return ls[0]["audio"].unsqueeze(0), ls[0]["idx"].item()

TRAIN_BATCH_SIZE = 1 # highly recommended to be 1
VALID_BATCH_SIZE = 1

train_dataset = DataLoader(tedlium_train, batch_size=TRAIN_BATCH_SIZE, collate_fn=collate_idx)
validation_dataset = DataLoader(tedlium_validation, batch_size=VALID_BATCH_SIZE, collate_fn=collate_idx)
test_dataset = DataLoader(tedlium_test)

# Training Loop

In [15]:
gc.collect()
torch.cuda.empty_cache()
gpu.print_cuda_usage()

0.14174842834472656 GB


In [16]:
LR = 1e-3
PATIENCE = 5
MIN_LIMIT = 45
ITER_LIMIT = 30
CLAMP_EP = 0.005
SNIPPET_SIZE = (1, 6_800)
POSITION = 16_000
PREPARE_METHOD = PrepareAtPosition(SNIPPET_SIZE, POSITION)

writer = None

In [19]:
# tensorboard writer
timestamp = datetime.datetime.now().strftime(f'%Y%m%d-%H%M%S_size_{SNIPPET_SIZE}_{PREPARE_METHOD.name}')
writer = SummaryWriter(log_dir=f"../runs/size_tests/{timestamp}", max_queue=5)

In [27]:
best_snippet, snippets, _, _ = gradient.train(model, train_dataset, validation_dataset, 
                                                            PREPARE_METHOD,
                                                            writer, lr=LR, 
                                                            train_success=None, valid_success=None,
                                                            iter_limit=ITER_LIMIT, mins_limit=MIN_LIMIT, patience=PATIENCE, clamp_epsilon=CLAMP_EP)

Prepare method: prepare_at_position
Snippet initialised to [3.107941438429407e-06, 0.004998547025024891] of size (1, 6800)
Clamp: 0.005
Time Limit (Mins): 45
Epochs Limit: 30
Tracking training success: False
Tracking valid success: False


Training:   0% 0/1 [03:01<?, ?it/s, Iter 1, Validation Batch 150/150]

Trng Avg Loss: 7.948866844177246 | Valid Avg Loss: 8.048896789550781 | Patience: 5 | LR: [0.001] | Epoch Limit: 29


Training:   0% 0/1 [05:05<?, ?it/s, Iter 2, Validation Batch 150/150]

Trng Avg Loss: 7.6776909828186035 | Valid Avg Loss: 7.407385349273682 | Patience: 5 | LR: [0.001] | Epoch Limit: 28


...

Trng Avg Loss: 0.28399837017059326 | Valid Avg Loss: 0.5657040476799011 | Patience: 5 | LR: [6.25e-05] | Epoch Limit: 9


Training:   0% 0/1 [36:55<?, ?it/s, Iter 23, Training Batch 1/500]    

Trng Avg Loss: 0.2702611982822418 | Valid Avg Loss: 0.5914958715438843 | Patience: 4 | LR: [6.25e-05] | Epoch Limit: 8


Training:   0% 0/1 [38:22<?, ?it/s, Iter 23, Training Batch 468/500]


Cleared buffer
Cleared loss
0.2993779182434082 GB


In [ ]:
audio.view_mel(best_snippet.detach().to("cpu").squeeze())

# Evaluation

In [28]:
gradient.evaluate(best_snippet, PREPARE_METHOD, model, test_dataset, CLAMP_EP, POSITION) # commented to prevent the runtime from autorunning and crashing the thing

[autoreload of utils.gradient failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 848, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/jovyan/code/utils/gradient.py", line 257, in <module>
    def evaluate(model: whisper.model.Whisper, snippet: Tensor, prepare_method: PrepareMethod, test_dataset: Dataset, clamp_ep: float, position: tuple):
NameError: name 'Dataset' is not defined
]
[autoreload of utils.gradient failed: Traceback (most recent cal

Clamp: 0.005
Prepare Method: prepare_at_position
Snippet Size: (1, 6800)
Position: 16000


Inference: 100%|██████████| 250/250 [01:10<00:00,  3.56it/s, Valid Examples: 194 | Empty Sequences: 113 | Total SL = 9166]



Total valid examples: 194
Success rate (Empty): 0.5824742268041238
Success rate (ASL): 47.24742268041237 (attacked) out of 122.16494845360825 (original)


In [ ]:
snippets = torch.stack(list(map(lambda x: x.cpu(), snippets)) + [best_snippet.cpu()])
snippets.shape

In [ ]:
torch.save(snippets.squeeze(), "snippets.pt")

In [ ]:
torch.save(torch.stack(list(map(torch.tensor, train_success.values()))), "train_success.pt")
torch.save(torch.tensor(list(train_success.keys())), "train_ids.pt")

In [ ]:
torch.save(torch.stack(list(map(torch.tensor, valid_success.values()))), "valid_success.pt")
torch.save(torch.tensor(list(valid_success.keys())), "valid_ids.pt")

# Save and Hear Snippet

In [ ]:
def normalise(random_snippet, ep):
    # we assume torch.rand inits to [0, 1)
    res = random_snippet * ep * 2 - ep
    print(f"Normalised, Min {torch.min(res)}, Max {torch.max(res)}")
    return res

In [ ]:
# Save snippet to wav file
save_audio(snippet, f"./snippets/clamp_{CLAMP_EP}_{PREPARE_METHOD.name}_snippet_only.wav")

In [ ]:
save_audio(PREPARE_METHOD(snippet.to("cpu"), tedlium_test[2]["audio"].unsqueeze(0)), f"./snippets/clamp_{CLAMP_EP}_{PREPARE_METHOD.name}_combined.wav")